In [ ]:
import pandas as pd
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from grad_diff import DualDataset, custom_gd_collator_forget, GradDiffTrainer
from config import Config
from peft import PeftModel, LoraConfig, get_peft_model
from perplexity import Perplexity

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,4"

In [3]:
## using the llama 3 template here, we can later change it to Olmo's template for our experiments

LLAMA3_CHAT_TEMPLATE = """<|start_header_id|>user<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

### Dataset & Finetuning

In [4]:
forget = pd.read_csv('/home/praveen/theoden/ul_paper/dataset/forget.csv')
retain = pd.read_csv('/home/praveen/theoden/ul_paper/dataset/retain.csv')

In [4]:
cfg = Config()

In [5]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model_id, token = cfg.access_token)
tokenizer.pad_token = tokenizer.eos_token

#### LoRA Finetuning

In [ ]:
model = AutoModelForCausalLM.from_pretrained(cfg.model_id, 
                                             device_map = 'auto',
                                             torch_dtype = torch.bfloat16, 
                                             token=cfg.access_token,)

In [11]:
config = LoraConfig(
        r = cfg.LoRA_r,
        lora_alpha = cfg.LoRA_alpha,
        lora_dropout= cfg.LoRA_dropout,
        target_modules = ['v_proj', 'k_proj', 'up_proj', 'o_proj', 'gate_proj', ' q_proj', 'down_proj'],
        bias = 'none',
        task_type = 'CAUSAL_LM',
    )
# wrapping the model with the LoRA configuration
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 18,874,368 || all params: 8,049,135,616 || trainable%: 0.2345


In [13]:
dataset = DualDataset(forget, retain, tokenizer, 266)


In [14]:
sample = dataset[2]
print(sample)

((tensor([128000, 128006,    882, 128007,    271,    678,   1403,  27373,  12631,
           304,    902,   8563,   1611,    452,   8869,  59335,     13, 128009,
        128006,  78191, 128007,    271,      1,  32449,     72,  14919,      1,
           323,    330,     49,   4210,  22353,   1210, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
        128009, 128009, 128009, 128009

In [16]:
# training arguments

training_args = TrainingArguments(
    output_dir = '/home/praveen/theoden/ul_paper/outputs/grad_diff',
    learning_rate = cfg.lr,
    per_device_train_batch_size= 4,
    num_train_epochs= 10,
    weight_decay = cfg.weight_decay,
    logging_dir = f'{cfg.save_dir}/logs',
    #save_steps = cfg.forget.save_steps,
    evaluation_strategy= 'no',
    save_total_limit= 2,
    bf16 = True,

)

/home/praveen/miniconda3/envs/semeval/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = GradDiffTrainer(
    model = model,
    args = training_args,
    train_dataset = dataset,
    tokenizer = tokenizer,
    data_collator = custom_gd_collator_forget,
)

/tmp/ipykernel_2227744/3049538443.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GradDiffTrainer.__init__`. Use `processing_class` instead.
  trainer = GradDiffTrainer(


In [18]:
# train the model
model.config.use_cache = False
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: praveenbushipaka942. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
500,-41.710600
1000,-177.752700


TrainOutput(global_step=1210, training_loss=-125.75792387654958, metrics={'train_runtime': 777.8972, 'train_samples_per_second': 6.183, 'train_steps_per_second': 1.555, 'total_flos': 0.0, 'train_loss': -125.75792387654958, 'epoch': 10.0})

In [19]:
output_dir = '/home/praveen/theoden/ul_paper/outputs/grad_diff'

model.save_pretrained(output_dir)
#tokenizer.save_pretrained(output_dir)
print(f'Forget LoRA adapter saved at {output_dir}')

Forget LoRA adapter saved at /home/praveen/theoden/ul_paper/outputs/grad_diff


In [20]:
del model
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [21]:
base_model = AutoModelForCausalLM.from_pretrained(cfg.model_id,
                                             device_map = 'auto',
                                             torch_dtype = torch.bfloat16,
                                             token = cfg.access_token)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
#config = PeftConfig.from_pretrained(output_dir)

peft_model_id = '/home/praveen/theoden/ul_paper/outputs/grad_diff'
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_and_unload()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

### perplexity

In [24]:
## perplexity on next token on forget set

batch_size = cfg.batch_size
max_length = 266

next_token_perplexity_ul = Perplexity(
    model = model, 
    tokenizer = tokenizer, 
    template = LLAMA3_CHAT_TEMPLATE, 
    batch_size = batch_size, 
    max_length = max_length,
    df =forget,
    case='next_token',
    chat_tokens=4)

print(next_token_perplexity_ul)

calculating perplexity for next_token! Please change this if this is not the case
Average loss for 16 batches: 169.24699020385742
tensor(inf)


In [26]:
## perplexity on retain after finetuning on gradient diff
## on next token prediction -> includes question and asnwer

batch_size = cfg.batch_size
max_length = 266

next_token_perplexity_ul = Perplexity(
    model = model, 
    tokenizer = tokenizer, 
    template = LLAMA3_CHAT_TEMPLATE, 
    batch_size = batch_size, 
    max_length = max_length,
    df =retain,
    case='next_token',
    chat_tokens=4)

print(next_token_perplexity_ul)

calculating perplexity for next_token! Please change this if this is not the case
Average loss for 9 batches: 39.25026363796658
tensor(1.1122e+17)


In [25]:
## perplexity on forget set after finetuning with gadient diff

## -> conditional perplexity calculation on answer given a question
qa_perplexity_ul = Perplexity(
    model = model, 
    tokenizer =tokenizer, 
    template =LLAMA3_CHAT_TEMPLATE, 
    batch_size =batch_size, 
    max_length =max_length,
    df =forget,
    case='qa',
    chat_tokens=4)

print(qa_perplexity_ul)

calculating perplexity for qa! Please change this if this is not the case
Average loss for 16 batches: 248.79578590393066
tensor(inf)


In [27]:
## perplexity on retain after finetuning on gradient diff
## -> conditional perplexity calculation on answer given a question

qa_perplexity_ul = Perplexity(
    model = model, 
    tokenizer =tokenizer, 
    template =LLAMA3_CHAT_TEMPLATE, 
    batch_size =batch_size, 
    max_length =max_length,
    df = retain,
    case='qa',
    chat_tokens=4)

print(qa_perplexity_ul)

calculating perplexity for qa! Please change this if this is not the case
Average loss for 9 batches: 48.369163301255966
tensor(1.0150e+21)


### changing forget to retain

In [13]:
# careful, im importing forget as retain file (because retain has less data)

retain = pd.read_csv('/home/praveen/theoden/ul_paper/dataset/forget.csv')
forget = pd.read_csv('/home/praveen/theoden/ul_paper/dataset/retain.csv')

In [17]:
dataset = DualDataset(forget, retain, tokenizer, 266)

In [18]:
# training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = '/home/praveen/theoden/ul_paper/outputs/grad_diff2',
    learning_rate = cfg.lr,
    per_device_train_batch_size= 4,
    num_train_epochs= 10,
    weight_decay = cfg.weight_decay,
    logging_dir = f'{cfg.save_dir}/logs',
    #save_steps = cfg.forget.save_steps,
    evaluation_strategy= 'no',
    save_total_limit= 2,
    bf16 = True,

)

/home/praveen/miniconda3/envs/semeval/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = GradDiffTrainer(
    model = model,
    args = training_args,
    train_dataset = dataset,
    tokenizer = tokenizer,
    data_collator = custom_data_collator_forget,
)

/tmp/ipykernel_2424033/3049538443.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GradDiffTrainer.__init__`. Use `processing_class` instead.
  trainer = GradDiffTrainer(


In [20]:
# train the model
model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: praveenbushipaka942. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
500,-18.872100
1000,-78.377100


TrainOutput(global_step=1210, training_loss=-56.120172068698345, metrics={'train_runtime': 773.1528, 'train_samples_per_second': 6.221, 'train_steps_per_second': 1.565, 'total_flos': 0.0, 'train_loss': -56.120172068698345, 'epoch': 10.0})

In [22]:
output_dir = '/home/praveen/theoden/ul_paper/outputs/grad_diff2'

model.save_pretrained(output_dir)
#tokenizer.save_pretrained(output_dir)
print(f'Forget LoRA adapter saved at {output_dir}')

Forget LoRA adapter saved at /home/praveen/theoden/ul_paper/outputs/grad_diff2


In [23]:
del model
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [24]:
base_model = AutoModelForCausalLM.from_pretrained(cfg.model_id,
                                             device_map = 'auto',
                                             torch_dtype = torch.bfloat16,
                                             token = cfg.access_token)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
from peft import PeftModel

#config = PeftConfig.from_pretrained(output_dir)

peft_model_id = '/home/praveen/theoden/ul_paper/outputs/grad_diff2'
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_and_unload()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

### perplexity 2

In [27]:
## perplexity on next token on forget set

batch_size = cfg.batch_size
max_length = 266

next_token_perplexity_ul = Perplexity(
    model = model, 
    tokenizer = tokenizer, 
    template = LLAMA3_CHAT_TEMPLATE, 
    batch_size = batch_size, 
    max_length = max_length,
    df =forget,
    case='next_token',
    chat_tokens=4)

print(next_token_perplexity_ul)

calculating perplexity for next_token! Please change this if this is not the case
Average loss for 9 batches: 88.04030524359808
tensor(1.7196e+38)


In [28]:
## perplexity on forget set after finetuning with gadient diff

## -> conditional perplexity calculation on answer given a question
qa_perplexity_ul = Perplexity(
    model = model, 
    tokenizer =tokenizer, 
    template =LLAMA3_CHAT_TEMPLATE, 
    batch_size =batch_size, 
    max_length =max_length,
    df =forget,
    case='qa',
    chat_tokens=4)

print(qa_perplexity_ul)

calculating perplexity for qa! Please change this if this is not the case
Average loss for 9 batches: 103.56071387396918
tensor(inf)


In [29]:
## perplexity on retain after finetuning on gradient diff
## on next token prediction -> includes question and asnwer

batch_size = cfg.batch_size
max_length = 266

next_token_perplexity_ul = Perplexity(
    model = model, 
    tokenizer = tokenizer, 
    template = LLAMA3_CHAT_TEMPLATE, 
    batch_size = batch_size, 
    max_length = max_length,
    df =retain,
    case='next_token',
    chat_tokens=4)

print(next_token_perplexity_ul)

calculating perplexity for next_token! Please change this if this is not the case
Average loss for 16 batches: 14.92553260922432
tensor(3034424.5000)


In [30]:
qa_perplexity_ul = Perplexity(
    model = model, 
    tokenizer =tokenizer, 
    template =LLAMA3_CHAT_TEMPLATE, 
    batch_size =batch_size, 
    max_length =max_length,
    df = retain,
    case='qa',
    chat_tokens=4)

print(qa_perplexity_ul)

calculating perplexity for qa! Please change this if this is not the case
Average loss for 16 batches: 12.331904746592045
tensor(226818.2656)
